In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset


In [ ]:
def angles_legend(images_amount, name, itype, step, mus, sigmas, amps, norm):
    """
    :param images_amount: int
    :param name: str
    :param itype: str
    :param step: int
    :param mus: float
    :param sigmas: float
    :param amps: float
    :param norm: int
    :return: str
    """
    #
    # создание легенды распределения углов
    #

    mu1 = round(mus[0], 2)
    sigma1 = round(sigmas[0], 2)
    amp1 = round(amps[0], 2)

    mu2 = round(mus[1], 2)
    sigma2 = round(sigmas[1], 2)
    amp2 = round(amps[1], 2)

    val = round(norm, 4)

    border = '--------------\n'
    total_number = '\n количество углов ' + str(val)
    images_number = '\n количество снимков ' + str(images_amount)
    text_angle = '\n шаг угла ' + str(step) + ' градусов'

    moda1 = '\n mu1 = ' + str(mu1) + ' sigma1 = ' + str(sigma1) + ' amp1 = ' + str(amp1)
    moda2 = '\n mu2 = ' + str(mu2) + ' sigma2 = ' + str(sigma2) + ' amp2 = ' + str(amp2)

    legend = border + name + ' ' + itype + total_number + images_number + text_angle + moda1 + moda2

    return legend

In [ ]:
def angles_approx_save(folder, images, name, names, types_dict, step, save=True):
    """
    :param folder: str path to dir
    :param images: ndarray uint8 [[image1_class1,image2_class1,..],[image1_class2,image2_class2,..]..]
    :param names: list str [class_name1,class_name2,..]
    :param types: list str [class_type1,class_type2,..]
    :param step: scalar int [0,N]
    :param save: bool
    """
    #
    # вычисление и сохранение распределения углов для всех фотографий одного образца
    #

    texts = []
    xy_scatter = []
    xy_gauss = []
    xy_gauss_data = []

    if not os.path.exists(folder):
        os.mkdir(folder)

    for i, images_list in tqdm(enumerate(images)):
        all_original_angles = []

        for j, image in enumerate(images_list):
            original_angles = grainMark.get_angles(image)

            for angle in original_angles:
                all_original_angles.append(angle)

        angles, angles_set, dens_curve = grainStats.stats_preprocess(all_original_angles, step)

        x = angles_set.astype(np.float64)
        y = dens_curve

        norm = np.sum(y)
        y = y / norm

        (x_gauss, y_gauss), mus, sigmas, amps = grainApprox.bimodal_gauss_approx(x, y)

        text = grainGenerate.angles_legend(len(images_list), names[i], types_dict[names[i]], step, mus, sigmas,
                                           amps, norm)

        xy_gauss.append((x_gauss, y_gauss))
        xy_scatter.append((x, y))
        xy_gauss_data.append((
            (mus[0], sigmas[0], amps[0]),
            (mus[1], sigmas[1], amps[1])
        ))

        texts.append(text)

    if save:
        np.save(f'{folder}/' + CfgAnglesNames.values + f'{name}' + f'{step}.npy',
                np.array(xy_scatter, dtype=object))
        np.save(f'{folder}/' + CfgAnglesNames.approx + f'{name}' + f'{step}.npy', np.array(xy_gauss))
        np.save(f'{folder}/' + CfgAnglesNames.approx_data + f'{name}' + f'{step}.npy', np.array(xy_gauss_data))
        np.save(f'{folder}/' + CfgAnglesNames.legend + f'{name}' + f'{step}.npy', np.array(texts))

### Сгенерируем распределения углов, аппроксимируем их бимодальным гауссом и сохраним

In [ ]:
from src.utils import grainGenerate, GrainLogs, grainMark, grainStats, grainApprox
import numpy as np

In [ ]:

# images = np.load('../data/saved np/images_4x_preprocess.npy')
# images = np.load('artif.npy')

# images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('data/saved_np/images_10_mixer_preprocess.npy')
images = np.load('images_7_classes_mixer_preprocess.npy')
# names = np.load('metadata_left_(1500,1500,1)_preprocessed.npy')
# names = np.load('data/saved_np/metadata_10_mixer_preprocess.npy')
names = np.load('metadata_7_classes_mixer_preprocess.npy')
names = [name.split('\\')[-1] for name in names]

folder = 'data/'

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}

# types_dict = {'class_1': 'Type None'}

types_dict = {'2550-51': '2550 призм, кадр 51\n доля карбина 54.51%',
              '2550-52': '2550 призм, кадр 52\n доля карбина 50.75%',
              '2550-53': '2550 призм, кадр 53\n доля карбина 45.81%',
              '3400-51': '3400 призм, кадр 51\n доля карбина 53.53%',
              '3400-52': '3400 призм, кадр 52\n доля карбина 50.47%',
              '3400-53': '3400 призм, кадр 53\n доля карбина 45.88%',
              '3400-54': '3400 призм, кадр 54\n доля карбина 40.88%'}

dens_dict={'2550-51': 0.5451066158234127,
 '2550-52': 0.5075556821469908,
 '2550-53': 0.4580166661499669,
 '3400-51': 0.535331783234127,
 '3400-52': 0.504782962859623,
 '3400-53': 0.45885131190062833,
 '3400-54': 0.40882115988756607}

# path = 'data/saved np'
# path = 'data/saved np'
path = 'data/artif_31.01'
# name = 'artif'
# name = 'left_(1500,1500,1)'
name = '7_classes_mixer_preprocess'


# grain_names =['cube compress', 'magnet']
# types = ['-', '-', ]

for step in range(1, 6):
    grainGenerate.angles_approx_save(path, images, name, names, types_dict, step, save=True)


In [ ]:
def angles_approx(folder, name, step, width, height, plot_name=None, y_lim=None, save=False,indices=None, font_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    xy_scatter = np.load(f'{folder}/' + CfgAnglesNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/' + CfgAnglesNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_gauss_data = np.load(f'{folder}/' + CfgAnglesNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgAnglesNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)

    if indices is not None:
        xy_scatter = xy_scatter[indices]
        xy_gauss = xy_gauss[indices]
        xy_gauss_data = xy_gauss_data[indices]
        texts = texts[indices]

    plt.rcParams['font.size'] = '25'
    gs = gridspec.GridSpec(1, 1)
    plt.figure(figsize=(width, height))

    ax = plt.subplot(gs[0])
    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1

    markers = ['D','v', 'd','s',   'p', '*','P']
    colors = ['blue','orange', 'green','red', 'indigo']

    for i in range(len(xy_scatter)):
        ax.plot(xy_gauss[i][0], xy_gauss[i][1],color=colors[i])

    # ax.legend(texts, fontsize=14, loc=1)

    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax.scatter(xy_scatter[i][0], xy_scatter[i][1], marker=marker,color=colors[i])

    # plt.ylabel('p(x)', fontsize=font_size)
    # plt.xlabel('углы, градусы', fontsize=font_size)

    print(texts)

    x = [0,60,120,180,240,300,360]
    plt.xticks(x, x)

    if y_lim:
        plt.ylim(0, y_lim)
    # plt.autoscale(enable=True, axis='y')

    if save:
        plt.savefig(f'распределение_углов_{plot_name}_шаг_{step}',bbox_inches='tight')

    plt.show()

In [ ]:
images = np.load('data/saved_metadata/images_4x_preprocess.npy')

In [ ]:
images[0].shape

In [ ]:
# folder = 'data/'
path = 'data/generated_data/angles/'
# path = 'images/saved np/'
# path = 'data/artif_31.01/'

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}


step = 2
width = 12
height = 8
# name = '4x_data_border_fix_'
# name = 'artif'
name = 'left_(1500,1500,1)'
# name = '7_classes_mixer_preprocess'
y_lim = 0.024
# y_lim = 0.013
# y_lim = None
# save = False
save = True
plot_name = 'original'



# генерация распределения, аппроксимирование распределения и сохранение данных
angles_approx(path, name, step, width, height, plot_name=plot_name, y_lim=y_lim, save=save, indices=[2,0,3])
# angles_approx(path, name, step, width, height, plot_name=plot_name, y_lim=y_lim, save=save, indices=None)

In [ ]:
def angles_approx(folder, step, width, height, save=False, font_size=20):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    xy_scatter = np.load(f'{folder}/' + CfgAnglesNames.values + f'{step}.npy', allow_pickle=True)
    xy_gauss = np.load(f'{folder}/' + CfgAnglesNames.approx + f'{step}.npy', allow_pickle=True)
    xy_gauss_data = np.load(f'{folder}/' + CfgAnglesNames.approx_data + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgAnglesNames.legend + f'{step}.npy', allow_pickle=True)

    synth_angles = np.load(f'{folder}/wc-co.npy')
    synth_angles = np.concatenate([synth_angles, np.zeros((180))])

    plt.rcParams['font.size'] = '15'
    gs = gridspec.GridSpec(1, 1)
    plt.figure(figsize=(width, height))

    ax = plt.subplot(gs[0])
    # маркеры для одновременного вывода скаттера для разных классов
    # количество варкеров=количество классов-1
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax.plot(xy_gauss[i][0], xy_gauss[i][1])
        ax.scatter(xy_scatter[i][0], xy_scatter[i][1], marker=marker)

    ax.plot(synth_angles[::-1])

    texts = list(texts)
    texts.append('синтетические углы')
    ax.legend(texts, fontsize=14, loc=2)
    plt.ylabel('p(x)', fontsize=font_size)
    plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
    if save:
        plt.savefig(f'наложение {step}')

    plt.show()

In [ ]:
folder = 'data/generated_data/angles'
step = 2
width = 20
height = 20

# генерация распределения, аппроксимирование распределения и сохранение данных
angles_approx(folder, step, width, height, save=False)

In [ ]:
def angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height, font_size=25):
    #
    # аппроксимация каждой моды
    #
    xy_scatter_o = np.load(f'{folder}/xy_scatter_step_{step}.npy', allow_pickle=True)
    xy_gauss_o = np.load(f'{folder}/xy_gauss_step_{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/texts_step_{step}.npy', allow_pickle=True)

    xy_scatter1 = []
    xy_scatter2 = []

    for j, sc in enumerate(xy_scatter_o):
        x = sc[0]
        y = sc[1]
        xy_scatter1.append([[], []])
        xy_scatter2.append([[], []])
        for i, xi in enumerate(x):
            if xi <= stop1 and xi >= start1:
                xy_scatter1[j][0].append(x[i])
                xy_scatter1[j][1].append(y[i])

            if xi <= stop2 and xi >= start2:
                xy_scatter2[j][0].append(x[i])
                xy_scatter2[j][1].append(y[i])

    plt.rcParams['font.size'] = '25'
    markers = ['v', 's', 'D', 'd', 'p', '*']
    k = 2
    for i in range(len(xy_scatter_o)):
        plt.figure(figsize=(width, height))
        marker = markers[i]

        #    plt.scatter(xy_scatter_o[i][0],xy_scatter_o[i][1])

        x1 = np.array(xy_scatter1[i][0])
        y1 = xy_scatter1[i][1]

        x2 = np.array(xy_scatter2[i][0])
        y2 = xy_scatter2[i][1]

        mu1, sigma1, amp1 = grainApprox.gaussian_fit(x1, y1, mu=120)
        mu2, sigma2, amp2 = grainApprox.gaussian_fit(x2, y2, mu=225)

        x_gauss1 = np.arange(mu1 - 3 * sigma1, mu1 + 3 * sigma1)
        x_gauss2 = np.arange(mu2 - 3 * sigma2, mu2 + 3 * sigma2)

        y_gauss1 = grainStats.gaussian(x_gauss1, mu1, sigma1, amp1)
        y_gauss2 = grainStats.gaussian(x_gauss2, mu2, sigma2, amp2)

        #    plt.plot(x_gauss,y_gauss)
        plt.scatter(x1, y1, marker=marker)
        plt.plot(x_gauss1, y_gauss1)

        plt.scatter(x2, y2, marker=markers[i + 1])
        plt.plot(x_gauss2, y_gauss2)

        #  print('mu1=',mu1,' sigma1=',sigma1,'amp1=',amp1)
        #  print('mu2=',mu2,' sigma2=',sigma2,'amp2=',amp2)
        plt.legend(['moda1', 'moda2', 'approx1', texts[i]], fontsize=14, loc=1)
        plt.ylabel('p(x)', fontsize=font_size)
        plt.xlabel('угол связующего вещества, градусы', fontsize=font_size)
        # plt.ylim([0,max(np.log(xy_scatter[i][1][:-1]))])
    # plt.savefig('крупные моды отдельная аппрокс  ' + names[i])

    #  print(texts)

    #  plt.savefig('распределение углов')
    plt.show()

In [ ]:
folder = 'saved np'
width = 12
height = 15

step = 1

start = 340
stop = 360

start1 = 50
stop1 = 180

start2 = 180
stop2 = 300

angles_approx_modes(folder, step, start1, stop1, start2, stop2, width, height)